In [3]:
import pandas as pd
from collections import Counter
import spacy
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import joblib
from copy import deepcopy
from sklearn.model_selection import train_test_split
from models.models import *
from utils.utils import *


In [2]:
import sys
full_path_to_SRC = 'C:/Users/Pau/Desktop/PycharmProjects/xnap-project-matcad_grup_10'
sys.path.append(full_path_to_SRC)

In [8]:
dataset = joblib.load('../data' + "/processed_dataset.joblib")

In [9]:
data_loader = get_data_loader(dataset, 256)

In [10]:
for i, j in data_loader:
    print(i)

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Pau\anaconda3\envs\xnap-example\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\Pau\anaconda3\envs\xnap-example\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Pau\anaconda3\envs\xnap-example\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Pau\Desktop\PycharmProjects\xnap-project-matcad_grup_10\SRC\utils\utils.py", line 152, in __getitem__
    caption_vec += self.vocab.numericalize(caption, self.spacy_eng)
AttributeError: 'FlickrDataset' object has no attribute 'spacy_eng'


In [12]:
(dataset.df.groupby('image')['caption'].count() != 5) .sum()

0

In [23]:
train_pct = 0.2


In [24]:
to_train = int(dataset.df.shape[0]*train_pct)

while to_train % 5 != 0: # While some image still splits into train and test, take more images
    to_train += 1

In [25]:
to_train

8095

In [26]:
dataset.df.sort_values(by='image').iloc[:to_train]

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
8094,2356574282_5078f08b58.jpg,A woman in a dance costume is bending over bac...
8093,2356574282_5078f08b58.jpg,A woman in a belly dancing outfit bending over...
8092,2356574282_5078f08b58.jpg,A woman bending over backwards .
8091,2356574282_5078f08b58.jpg,A costumed woman with a sword does a backbend .


In [27]:
dataset.df.sort_values(by='image').iloc[to_train:]

,image,caption
8095,2356664078_4b1e6e465d.jpg,Three dogs are running side by side on the gra...
8096,2356664078_4b1e6e465d.jpg,"Three dogs , one Rottweiler with a tennis ball..."
8097,2356664078_4b1e6e465d.jpg,Three dogs running in a field .
8098,2356664078_4b1e6e465d.jpg,"Three dogs trot on a stretch of grass , and on..."
8099,2356664078_4b1e6e465d.jpg,Three dogs with tongues out running on the gra...
...,...,...
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .


# A PARTIR DE AQUI FUNCIONA

In [12]:
import os
import random
import wandb

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T

from train import *
from test import *
from utils.utils import *
from models.models import *
import multiprocessing

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .


In [84]:

DATA_LOCATION = '../data'
device = 'cuda:0'

transforms = T.Compose([
    T.Resize(226),
    T.RandomCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

config = dict(
    root_dir=DATA_LOCATION+"/Images",
    captions_file=DATA_LOCATION+"/captions.txt",
    device=device,
    encoder='ResNet50',
    transforms=transforms,
    embed_size=300,
    attention_dim=256,
    encoder_dim=2048,
    decoder_dim=512,
    epochs=25,
    learning_rate=3e-4,
    batch_size=int(256),
    DATA_LOCATION=DATA_LOCATION,
    train_size=0.8,
    save=True
)

def make_dataset(config):
    dataset = joblib.load(config['DATA_LOCATION'] + "/processed_dataset.joblib")
    dataset.spacy_eng = spacy.load("en_core_web_sm")
    return dataset

def make_dataloaders(config, dataset, num_workers):
    train_dataset, test_dataset = flickr_train_test_split(dataset, config['train_size'])

    train_loader = get_data_loader(train_dataset, batch_size=config['batch_size'], num_workers=num_workers)
    test_loader = get_data_loader(test_dataset, batch_size = 5, num_workers=num_workers)

    return train_loader, test_loader
dataset = make_dataset(config)

vocab = dataset.vocab

In [87]:

dataset.df = dataset.df.iloc[:1000]
train_loader, test_loader = make_dataloaders(config, dataset, 1)

In [35]:

t0 = time.time()
for i, j in train_loader:
    pass

t1 = time.time()
print(t1-t0)

10.11643099784851


In [90]:
train_loader.dataset.df

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
795,1177994172_10d143cb8d.jpg,"Two blonde boys , one in a camouflage shirt an..."
796,1177994172_10d143cb8d.jpg,Two boys are squirting water guns at each other .
797,1177994172_10d143cb8d.jpg,Two boys spraying each other with water
798,1177994172_10d143cb8d.jpg,Two children wearing jeans squirt water at eac...


In [91]:
# Preprocessing the data
data_list = []

for _, (img_name, caption) in dataset.df.iterrows():
    img_location = os.path.join(dataset.root_dir, img_name)
    img = Image.open(img_location)
    img = img.convert("RGB")
    
    img = dataset.transform(img)
     
    caption_vec = []
    caption_vec += [dataset.vocab.stoi["<SOS>"]]
    caption_vec += dataset.vocab.numericalize(caption, dataset.spacy_eng)
    caption_vec += [dataset.vocab.stoi["<EOS>"]]

    data_list.append( [img.to(torch.int16), torch.tensor(caption_vec).to(torch.float16)] )
    
# Split into train and test

pad_idx = dataset.vocab.stoi["<PAD>"]
collate_fn = CapsCollate(pad_idx=pad_idx, batch_first=True) # Fills the rest of the captions with padding

data_loader = DataLoader(
        dataset=data_list,
        batch_size=50,
    	collate_fn = collate_fn
    )

In [124]:
torch.tensor(data_list)

ValueError: only one element tensors can be converted to Python scalars

In [102]:
data_loader.dataset[0][1].dtype

torch.int64

In [107]:
data_loader.dataset[0][1].to(torch.int16)

tensor([  1,   4,  28,   8,   4, 195, 151,  17,  32,  67,   4, 353,  11, 711,
          8,  24,   3, 496,   5,   2], dtype=torch.int16)

In [110]:
data_loader.dataset[0][0].to(torch.float16)

tensor([[[-0.6108, -0.3027,  0.1255,  ..., -1.8271, -1.7930, -1.4160],
         [-0.3198, -0.6279, -0.8164,  ..., -1.7930, -1.7930, -1.6387],
         [-0.1315, -0.2341, -0.4910,  ..., -1.8438, -1.8955, -1.8438],
         ...,
         [-0.1315,  0.0227,  0.1768,  ...,  2.0605,  2.1113,  2.0605],
         [ 0.2281,  0.3481,  0.3137,  ...,  2.0781,  2.0957,  2.1113],
         [ 0.4165,  0.2966,  0.4509,  ...,  2.1465,  2.0957,  2.0430]],

        [[-1.0205, -0.7754, -0.3726,  ..., -1.5459, -1.5107, -0.9150],
         [-0.4951, -0.7051, -1.0908,  ..., -1.4229, -1.4580, -1.2305],
         [-0.3201, -0.3201, -0.7402,  ..., -1.3877, -1.4580, -1.4756],
         ...,
         [-0.3025, -0.2150, -0.0049,  ...,  2.3066,  2.3242,  2.2891],
         [ 0.0476,  0.1002,  0.1002,  ...,  2.3066,  2.3066,  2.3242],
         [ 0.1876,  0.0476,  0.1177,  ...,  2.2539,  2.2715,  2.2715]],

        [[-1.2295, -1.2295, -1.1250,  ..., -1.6123, -1.7178, -1.5781],
         [-0.4451, -0.7065, -1.0898,  ..., -1

In [120]:
len(data_loader.dataset[:])

1000

In [118]:
data_loader.dataset

AttributeError: 'list' object has no attribute 'shape'

In [80]:
t0 = time.time()

for i, j in data_loader:
    print(j)

t1 = time.time()
print(t1-t0)

tensor([[  1,   4,  28,  ...,   0,   0,   0],
        [  1,   4,   7,  ...,   0,   0,   0],
        [  1,   4,   9,  ...,   0,   0,   0],
        ...,
        [  1,   4,  53,  ...,   0,   0,   0],
        [  1,   4,  20,  ...,   0,   0,   0],
        [  1,   4, 251,  ...,   0,   0,   0]])
tensor([[ 1,  4, 30,  ...,  0,  0,  0],
        [ 1,  4,  6,  ...,  0,  0,  0],
        [ 1,  4,  6,  ...,  0,  0,  0],
        ...,
        [ 1,  4, 56,  ...,  0,  0,  0],
        [ 1,  4, 56,  ...,  0,  0,  0],
        [ 1, 50, 96,  ...,  0,  0,  0]])
tensor([[  1,  50, 321,  ...,   0,   0,   0],
        [  1,  50,  51,  ...,   0,   0,   0],
        [  1,  50,  51,  ...,   0,   0,   0],
        ...,
        [  1,   4,  14,  ...,   0,   0,   0],
        [  1, 102,  10,  ...,   0,   0,   0],
        [  1, 204,  17,  ...,   0,   0,   0]])
tensor([[ 1, 10, 43,  ...,  0,  0,  0],
        [ 1, 10, 43,  ...,  0,  0,  0],
        [ 1,  4, 85,  ...,  0,  0,  0],
        ...,
        [ 1,  4, 93,  ...,  0,  0

In [59]:
data_loader.dataset

AttributeError: 'list' object has no attribute 'shape'

In [86]:
dataset.df

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .
